In [2]:
import random
import string
import time
import psycopg2
from pymongo import MongoClient

In [3]:
def gerar_dados_aleatorios(num_registros):
    """
    Gera dados aleatórios para teste de performance
    """
    dados = []
    for i in range(num_registros):
        registro = {
            'id': i + 1,
            'nome': ''.join(random.choices(string.ascii_letters, k=10)),
            'idade': random.randint(18, 80),
            'email': ''.join(random.choices(string.ascii_letters, k=15)) + '@example.com'
        }
        dados.append(registro)
    return dados

def inserir_postgresql(dados):
    """
    Insere dados no PostgreSQL e mede o tempo de execução
    """
    try:
        conn = psycopg2.connect(
            dbname='testedb',
            user='postgres',
            password='univassouras',
            host='localhost'
        )
        cur = conn.cursor()
        
        # Criar tabela se não existir
        cur.execute('''
            CREATE TABLE IF NOT EXISTS usuarios (
                id SERIAL PRIMARY KEY,
                nome VARCHAR(50),
                idade INTEGER,
                email VARCHAR(100)
            )
        ''')
        
        # Limpar tabela para teste limpo
        cur.execute('TRUNCATE TABLE usuarios RESTART IDENTITY')
        
        inicio = time.time()
        
        # Inserção em lote para melhor performance
        dados_tupla = [(r['id'], r['nome'], r['idade'], r['email']) for r in dados]
        cur.executemany('INSERT INTO usuarios (id, nome, idade, email) VALUES (%s, %s, %s, %s)', dados_tupla)
        conn.commit()
        
        fim = time.time()
        
        cur.close()
        conn.close()
        
        return fim - inicio
        
    except Exception as e:
        print(f"Erro no PostgreSQL: {e}")
        return None

def inserir_mongodb(dados):
    """
    Insere dados no MongoDB e mede o tempo de execução
    """
    try:
        client = MongoClient('mongodb://localhost:27017/')
        db = client['testedb']
        collection = db['usuarios']
        
        # Limpar coleção anterior
        collection.drop()
        
        inicio = time.time()
        
        # Inserção em lote
        collection.insert_many(dados)
        
        fim = time.time()
        
        client.close()
        
        return fim - inicio
        
    except Exception as e:
        print(f"Erro no MongoDB: {e}")
        return None

def main():
    """
    Função principal que executa o teste de performance
    """
    # Gerar 1 milhão de registros
    print("Gerando 1 milhão de registros...")
    dados = gerar_dados_aleatorios(1000000)
    print(f"Dados gerados: {len(dados)} registros")
    print("-" * 50)
    
    # Executar teste PostgreSQL
    print("Testando PostgreSQL...")
    tempo_postgresql = inserir_postgresql(dados)
    
    # Executar teste MongoDB
    print("Testando MongoDB...")
    tempo_mongodb = inserir_mongodb(dados)
    
    # Apresentar resultados
    print("\n" + "="*50)
    print("RESULTADOS DO TESTE DE PERFORMANCE")
    print("="*50)
    
    if tempo_postgresql is not None:
        print(f'PostgreSQL: {tempo_postgresql:.2f} segundos')
    else:
        print('PostgreSQL: Falha na execução')
        
    if tempo_mongodb is not None:
        print(f'MongoDB: {tempo_mongodb:.2f} segundos')
    else:
        print('MongoDB: Falha na execução')
    
    # Comparação de performance
    if tempo_postgresql is not None and tempo_mongodb is not None:
        if tempo_postgresql < tempo_mongodb:
            diferenca = ((tempo_mongodb - tempo_postgresql) / tempo_postgresql) * 100
            print(f'\nPostgreSQL foi {diferenca:.1f}% mais rápido que MongoDB')
        else:
            diferenca = ((tempo_postgresql - tempo_mongodb) / tempo_mongodb) * 100
            print(f'\nMongoDB foi {diferenca:.1f}% mais rápido que PostgreSQL')

In [4]:
if __name__ == "__main__":
    main()

Gerando 1 milhão de registros...
Dados gerados: 1000000 registros
--------------------------------------------------
Testando PostgreSQL...
Testando MongoDB...

RESULTADOS DO TESTE DE PERFORMANCE
PostgreSQL: 41.77 segundos
MongoDB: 5.62 segundos

MongoDB foi 643.7% mais rápido que PostgreSQL
